In [85]:
import pickle
from pprint import pprint
import json
import gzip
fp = open("hello.objdump.pickle","rb")
programCounter = pickle.load(fp)
programCounter={int(k):v for k,v in programCounter.items()} 
fp.close()
fp = open("output_file","rb")
data_from_browser = fp.read().decode("utf-8")
fp.close()
all_data={}
index=0
# pprint(data_from_browser)
data_from_browser1 = data_from_browser.split("hello.html")[1].split("blockend")
data_from_browser1.pop()

output_dictionary={}
instruction_count=0
output_list=[]

# for i in data_from_browser1:
#     temp=json.loads(i)
#     temp = {int(k):v for k,v in temp.items()} 
#     for j in range(len(temp.keys())):
#         instruction_data = temp[j]
#         if 'op' in instruction_data.keys():
#             function_name = instruction_data['op']
#         else:
#             function_name = instruction_data['hook_name']
#         output_dictionary[instruction_count]={}
#         output_dictionary[instruction_count]["function_name"]=function_name
#         output_dictionary[instruction_count]["function_number"]=instruction_data["location"]["func"]
#         output_dictionary[instruction_count]["instruction_number"]=instruction_data["location"]["instr"]
#         output_dictionary[instruction_count]["base_address"]=programCounter[instruction_data["location"]["func"]]['base_address']
#         output_dictionary[instruction_count]["programCounter"]=int(programCounter[instruction_data["location"]["func"]]['instructions'][instruction_data["location"]["instr"]],16)
#         output_list.append(str(output_dictionary[instruction_count]["programCounter"])+"\t"+str(output_dictionary[instruction_count]["function_name"])+"\n")
#         instruction_count+=1

# pprint(data_from_browser1)

In [1]:
import pickle
from pprint import pprint
import json
import gzip
fp = open("hello.objdump.pickle","rb")
programCounter = pickle.load(fp)
programCounter={int(k):v for k,v in programCounter.items()} 
fp.close()
fp = open("output_file","rb")
data_from_browser = fp.read().decode("utf-8")
fp.close()
all_data={}
index=0
# pprint(data_from_browser)
data_from_browser1 = data_from_browser.split("hello.html")[1].split("blockend")
data_from_browser1.pop()

output_dictionary={}
instruction_count=0
output_list=[]
def get_end(fun_num, instr_num):
    flag = -1
    dump_file = open("hello.objdump.txt")
    dump_lines = dump_file.readlines()
    i = 0
    while i < (len(dump_lines)):
        lines = dump_lines[i]
        if ("func[" + str(fun_num) + "]") in lines:
            i = i + 1
            lines = dump_lines[i]
            while("local[" in lines):
                i = i + 1
                lines = dump_lines[i]
            i = i + instr_num
            lines = dump_lines[i]
            
            if("if" not in lines):
                print(lines)
                raise Exception('Not if')
            for j in range(len(lines)):
                if(lines[j]=='i' and lines[j+1]=='f'):
                    flag = j
                    break
        elif(flag>-1):
            if("end" in lines or "else" in lines):
                if(lines[flag:flag+3]=='end' or lines[flag:flag+4]=='else'):
                    return int("0x" + str(dump_lines[i+1].split(":")[0].strip()),16)
        i = i + 1

data_warehouse = {}
count = 0
argument_wise_data = {}
for i in data_from_browser1:
    temp=json.loads(i)
    temp = {int(k):v for k,v in temp.items()} 
    for key,value in temp.items():
        data_warehouse[count] = value
        count+=1
count=0
for key,value in data_warehouse.items():
    if value['hook_name']=='start' or value['hook_name']=='nop' or value['hook_name']=='unreachable' :
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count] = str(int(pc,16))+" 27 "+value['hook_name']
    elif value['hook_name']== 'if_':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        if value['condition']== False:
            nextval= data_warehouse[key+1]
            pc2=programCounter[nextval['location']['func']]['instructions'][nextval['location']['instr']]
            argument_wise_data[count] = str(int(pc,16))+" 27 "+"if " +str(pc2)
            count+=1
            argument_wise_data[count] = str(int(pc,16))+" 4 "+str(pc2)
        else:
            pc2 = get_end(value['location']['func'],value['location']['instr'])
            argument_wise_data[count] = str(int(pc,16))+" 27 "+"if " +str(pc2)
            argument_wise_data[count] = str(int(pc,16))+" 5 "+str(pc2)
    
    elif value['hook_name']== 'br_if':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        if value['condition']== False:
            nextval= data_warehouse[key+1]
            pc2=programCounter[nextval['location']['func']]['instructions'][nextval['location']['instr']]
            argument_wise_data[count] = str(int(pc,16))+" 27 "+"br_if " +str(pc2)
            count+=1
            argument_wise_data[count] = str(int(pc,16))+" 5 "+str(pc2)
        else:
            pc2 = get_end(value['location']['func'],value['location']['instr'])
            argument_wise_data[count] = str(int(pc,16))+" 27 "+"if " +str(pc2)
            argument_wise_data[count] = str(int(pc,16))+" 4 "+str(pc2)
  
    elif value['hook_name']== 'br':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count]= str(int(pc,16))+" 27 "+value['hook_name']+ " "+str(value['target']['label'])
            
    elif value['hook_name']=='local':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count] = str(int(pc,16))+" 27 "+value['op']+ " " +str(value['localIndex']) + " "+str(value['value'])
    elif value['hook_name']=='global':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count] = str(int(pc,16))+" 27 "+value['op']+  " " +str(value['globalIndex']) + " "+ str(value['value'])
    elif value['hook_name']=='load':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count] = str(int(pc,16))+" 27 "+ value['op'] + " " +str( value['memarg']['align'] )+ " "+str(value['memarg']['offset'])
        count+=1
        argument_wise_data[count] = str(int(pc,16))+" 2 " + str(value['memarg']['addr'])
  
    elif value['hook_name']=='store':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count] = str(int(pc,16))+" 27 "+ value['op'] + " " +str( value['memarg']['align'] )+ " "+str(value['memarg']['offset'])
        count+=1
        argument_wise_data[count] = str(int(pc,16))+" 3 " + str(value['memarg']['addr'])
    elif value['hook_name']=='binary':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count] = str(int(pc,16))+" 27 "+value['op'] + " " +str( value['result'] )+ " " +str(value['first'])+ " " +str(value['second'])
    elif value['hook_name']=='unary':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count] =str(int(pc,16))+" 27 "+ value['op'] + " " +str( value['result'] )+ " " +str(value['input'])
    elif value['hook_name']=='const_':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count] = str(int(pc,16))+" 27 "+value['op'] + " " +str( value['value'] )
    elif value['hook_name']=='return_':
        if value['location']['instr']==-1:
            continue
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count] = str(int(pc,16))+" 27 "+"return"+ " " +str( value['values'])
    elif value['hook_name'] =='end':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count] = str(int(pc,16))+" 27 "+value['hook_name']
    elif value['hook_name']=='call_pre':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count]= str(int(pc,16))+" 27 "+"call"+ " "+str(value['target'])
    

                
    elif value['hook_name']=='select':
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        if value['cond'] ==True:
            argument_wise_data[count]= str(int(pc,16))+" 27 "+"i32.const "+str(value['first'])
        else:
             argument_wise_data[count]= str(int(pc,16))+" 27 "+"i32.const "+str(value['second'])
    elif value['hook_name']=='begin':
        if value['type']!='if' and value['type']!='function' and value['type']!='block':
            pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
            argument_wise_data[count] = str(int(pc,16))+" 27 "+value['hook_name']
        else:
            argument_wise_data[count] = str(int(pc,16))+" 27 "+"NOP"
            
    else:
        pc = programCounter[value['location']['func']]['instructions'][value['location']['instr']]
        argument_wise_data[count] = str(int(pc,16))+" 27 "+"NOP"
        
    count+=1
pprint(data_warehouse)   
# with open('data.pickle', 'wb') as handle:
#     pickle.dump(argument_wise_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

data = argument_wise_data

FileNotFoundError: [Errno 2] No such file or directory: 'hello.objdump.pickle'

In [101]:
# dada=open('data.pickle','rb')
# data=pickle.load(dada)
# pprint(data)
mpmp=open('mapX86.pickle','rb')
mapx86=pickle.load(mpmp)
# pprint(maop)
# pprint(maop['i32.const'])




In [102]:
for key,value in data.items():
    insList= value.split(" ")
#     print(insList)
    try:
        insList[2] = mapx86[insList[2].strip()]
    except KeyError:
         continue
    data[key]= " ".join(insList)

pprint(data)

{0: '15217 27 NOP',
 1: '20669 27 GET 1 5104',
 2: '20671 27 MOV 2 5104',
 3: '20673 27 MOV 0 8',
 4: '20675 27 GET 1 5104',
 5: '20677 27 ADD 5112 8 5104',
 6: '20678 27 SET 1 5112',
 7: '20680 27 GET 1 5112',
 8: '20682 27 EQU 15',
 9: '20684 27 ADD 5127 5112 15',
 10: '20685 27 EQU -16',
 11: '20687 27 AND 5120 5127 -16',
 12: '20688 27 SET 1 5120',
 13: '20690 27 MOV 2 5104',
 14: '20692 27 END',
 15: '20692 27 NOP',
 16: '20669 27 GET 1 5120',
 17: '20671 27 MOV 2 5120',
 18: '20673 27 MOV 0 15',
 19: '20675 27 GET 1 5120',
 20: '20677 27 ADD 5135 15 5120',
 21: '20678 27 SET 1 5135',
 22: '20680 27 GET 1 5135',
 23: '20682 27 EQU 15',
 24: '20684 27 ADD 5150 5135 15',
 25: '20685 27 EQU -16',
 26: '20687 27 AND 5136 5150 -16',
 27: '20688 27 SET 1 5136',
 28: '20690 27 MOV 2 5120',
 29: '20692 27 END',
 30: '20692 27 NOP',
 31: '15028 27 GET 1 5136',
 32: '15030 27 MOV 5 5136',
 33: '15032 27 GET 1 5136',
 34: '15034 27 EQU 32',
 35: '15036 27 ADD 5168 5136 32',
 36: '15037 27 SE

 1393: '15122 27 ADD 4 3 1',
 1394: '15123 27 TEE 2 4',
 1395: '15125 27 EQU 30',
 1396: '15127 27 FICOM 1 4 30',
 1397: '15128 4 None',
 1398: '15130 27 END',
 1399: '15047 27 begin',
 1400: '15049 27 MOV 2 4',
 1401: '15051 27 EQU 3',
 1402: '15053 27 SHL 32 4 3',
 1403: '15054 27 MOV 3 5248032',
 1404: '15056 27 ADD 5248064 32 5248032',
 1405: '15057 27 MOV 2 4',
 1406: '15059 27 VCVTUSI2SS 4 4',
 1407: '15060 27 TEE 7 4',
 1408: '15062 27 EQU 2',
 1409: '15071 27 FADD 6 4 2',
 1410: '15072 27 EQU 30',
 1411: '15081 27 FDIV  0.2 6 30',
 1412: '15082 27 f64.store 3 0',
 1413: '15082 3 5248064',
 1414: '15085 27 MOV 2 4',
 1415: '15087 27 EQU 3',
 1416: '15089 27 SHL 32 4 3',
 1417: '15090 27 MOV 4 5248288',
 1418: '15092 27 ADD 5248320 32 5248288',
 1419: '15093 27 MOV 7 4',
 1420: '15095 27 EQU 3',
 1421: '15104 27 FADD 7 4 3',
 1422: '15105 27 EQU 30',
 1423: '15114 27 FDIV  0.23333333333333334 7 30',
 1424: '15115 27 f64.store 3 0',
 1425: '15115 3 5248320',
 1426: '15118 27 MOV 2

 2641: '15180 27 EQU 3',
 2642: '15182 27 SHL 72 9 3',
 2643: '15183 27 MOV 3 5248032',
 2644: '15185 27 ADD 5248104 72 5248032',
 2645: '15186 27 f64.load 3 0',
 2646: '15186 2 5248104',
 2647: '15189 27 FADD 1 0.6333333333333333 0.36666666666666664',
 2648: '15190 27 EQU 0.33333',
 2649: '15199 27 FMUL 0.33333 1 0.33333',
 2650: '15200 27 f64.store 3 0',
 2651: '15200 3 5248352',
 2652: '15203 27 MOV 2 9',
 2653: '15205 27 EQU 29',
 2654: '15207 27 FICOM 1 9 29',
 2655: '15208 4 None',
 2656: '15210 27 END',
 2657: '15137 27 begin',
 2658: '15139 27 MOV 2 9',
 2659: '15141 27 EQU 3',
 2660: '15143 27 SHL 72 9 3',
 2661: '15144 27 MOV 4 5248288',
 2662: '15146 27 ADD 5248360 72 5248288',
 2663: '15147 27 MOV 2 9',
 2664: '15149 27 EQU -1',
 2665: '15151 27 ADD 8 9 -1',
 2666: '15152 27 EQU 3',
 2667: '15154 27 SHL 64 8 3',
 2668: '15155 27 MOV 3 5248032',
 2669: '15157 27 ADD 5248096 64 5248032',
 2670: '15158 27 f64.load 3 0',
 2671: '15158 2 5248096',
 2672: '15161 27 MOV 2 9',
 267